<div style="direction:rtl; text-align:right">

# المشروع السادس: هل تستطيع تصنيف النصوص؟

<div style="direction:rtl; text-align:right">
أهلاً بكم في المشروع السادس من علم البيانات. في هذا المشروع، سنقوم معاً بالنظر إلى بيانات فريدة من نوعها وهي مجموعة من النصوص المصنفة إلى مجالات مختلفة. سنقوم بنمذجة هذه البيانات لتوقع تصنيف النصوص بناء على الكلمات الواردة. تستطيع قراءة المزيد عن البيانات من خلال موقع سايكت-ليرن:

- https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html

<div style="direction:rtl; text-align:right">
ملاحظة: عليك تعديل الخانات التي يوجد فيها الكود أدناه قبل تسليم المشروع. جميع الخانات الأخرى يجب ان تبقى كما هي بدون أي 
تعديل.

```
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################
```

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

<div style="direction:rtl; text-align:right">

# قراءة البيانات

In [2]:
from sklearn.datasets import fetch_20newsgroups

In [3]:
newsgroups_data = fetch_20newsgroups(subset='train')

In [4]:
X = newsgroups_data['data']
y = newsgroups_data['target']

<div style="direction:rtl; text-align:right">
لاحظ أن X هنا عبارة عن قائمة بالنصوص، وليست جدول كما هو الحال مع الدروس الأخرى. أما y فهي كذلك قائمة بالتصنيفات المختلفة. لمعرفة معنى التصنيفات نستطيع طباعة target_names:

In [5]:
for i, k in enumerate(newsgroups_data['target_names']):
    print(i, '\t->\t', k)

0 	->	 alt.atheism
1 	->	 comp.graphics
2 	->	 comp.os.ms-windows.misc
3 	->	 comp.sys.ibm.pc.hardware
4 	->	 comp.sys.mac.hardware
5 	->	 comp.windows.x
6 	->	 misc.forsale
7 	->	 rec.autos
8 	->	 rec.motorcycles
9 	->	 rec.sport.baseball
10 	->	 rec.sport.hockey
11 	->	 sci.crypt
12 	->	 sci.electronics
13 	->	 sci.med
14 	->	 sci.space
15 	->	 soc.religion.christian
16 	->	 talk.politics.guns
17 	->	 talk.politics.mideast
18 	->	 talk.politics.misc
19 	->	 talk.religion.misc


<div style="direction:rtl; text-align:right">
وهنا سنقوم بطباعة أول قيمة من X

In [6]:
print(X[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







<div style="direction:rtl; text-align:right">
لنقم أولاً بعرض كم عدد النصوص لكل صنف ؟ قد تحتاج لاستخدام باندا للقيام بالمهمة!

In [7]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################
dd=pd.Series(y)
dd.value_counts()

10    600
15    599
8     598
9     597
11    595
13    594
7     594
14    593
5     593
12    591
2     591
3     590
6     585
1     584
4     578
17    564
16    546
0     480
18    465
19    377
dtype: int64

<div style="direction:rtl; text-align:right">
نرى أن لدينا توازن في توزيع النصوص بين الأصناف المختلفة

<div style="direction:rtl; text-align:right">

# تجهيز البيانات

<div style="direction:rtl; text-align:right">
ماذا نعمل عادة مع البيانات النصية؟ في هذه الحالة، سيكون أمامنا الكثير لنعمله حتى نقوم بتنظيف هذه البيانات وتهيئتها للتحليل. لذلك سنقوم بكتابة بعض الأوامر التي تساعدنا على ذلك.

<div style="direction:rtl; text-align:right">
المهمة الأولى: تنظيف النص

In [8]:
import re

In [9]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################


# HINT: figure out how to: delete special charachters, delete numbers, delete email addresses
#          if you dont know, use google!
import string

def clean_text(text):
    text = text.lower()
    text = re.sub(re.compile(r"[-a-z0-9_.]+.(?:[-a-z0-9]+\.)+[a-z]{2,6}", re.IGNORECASE), '', text)
    text = re.sub(re.compile(r"[-a-z0-9_.]+@(?:[-a-z0-9]+\.)+[a-z]{2,6}", re.IGNORECASE), '', text)
    text = re.sub(re.compile(r'[%s]' % re.escape(string.punctuation), re.IGNORECASE), ' ', text)
    text = re.sub(r'[@;.:,%!?1234567890.]+', ' ', text)
    text = re.sub(re.compile(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))',re.IGNORECASE) , ' ', text)
    text = text.lower()
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub('[‘’“”…]', ' ', text)
    text = re.sub(' +', ' ', text)
    text = re.sub('\n', '', text)
    text = re.sub('\t', '', text)
    text = text.strip()
    text = re.sub(r"http\S+", "", text)
    return text

<div style="direction:rtl; text-align:right">
حسنا، سنقوم الآن برؤية أثر هذه التعديلات على العنصر الأول في قائمة النصوص

In [10]:
print(clean_text(X[10]))

from irwin arnstein subject re recommendation on ducsummary what s it worth distribution usaexpires sat may gmtorganization computrac inc richardson txkeywords ducati gts how much lines i have a line on a ducati gts model with k on the clock runsvery well paint is the bronze brown orange faded out leaks a bit of oiland pops out of st with hard accel the shop will fix trans and oil leak they sold the bike to the and only owner they want andi am thinking more like k any opinions out there please email me thanks it would be a nice stable mate to the beemer then i ll geta jap bike and call myself axis motors    tuba irwin i honk therefore i am computrac richardson tx dod r


<div style="direction:rtl; text-align:right">
سنقوم الآن بتنفيذ ذات الأمر على باقي النصوص في X:

In [11]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################
t_en= []
for  text in X:
    cleaned = clean_text(text)
    t_en.append(cleaned)
X = t_en

<div style="direction:rtl; text-align:right">
الآن سنقوم باستخدام Td-IDF لتحويل الكلمات إلى مصفوفة ضخمة حتى نستخدمها في النمذجة. في الخانة التالية، ستقوم باستخدام TfidfVectorizer لتحويل النص.

In [12]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()
trn_term_doc = vec.fit_transform(X)

In [13]:
trn_term_doc.shape

(11314, 141400)

<div style="direction:rtl; text-align:right">

# نمذجة البيانات

<div style="direction:rtl; text-align:right">
سنقوم الآن بنمذجة البيانات. لنبدأ بالانحدار اللوجستي. في الخانة أدناه، قم باستيراد كلاً من مكتبة الانحدار اللوجستي، وأمر cross_val_score ثم استخدمهم في تدريب النموذج والتحقق من صحته.

In [14]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################

# HINT: You should use `train_data` as your input
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

model = LogisticRegression()
scores = cross_val_score(model, trn_term_doc, y, scoring='accuracy')

In [15]:
avg_score = scores.mean()

In [16]:
print('Average accuracy is: ', avg_score)

Average accuracy is:  0.8791758238967466


<div style="direction:rtl; text-align:right">
لا تحتاج في هذا المشروع إلى تسليم أي ملف إلى كاقل. ولكن تأكد من صحة الإجابات ثم قم بتسليم المشروع في الموقع.